In [14]:
import numpy as np 
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
from random import shuffle
from keras.utils  import to_categorical
import pickle

In [15]:
def load_rand():
    X=[]
    dir_path=r'C:\Users\fadel\Documents\datasets\dataset\train'
    for sub_dir in tqdm(os.listdir(dir_path)):
        print(sub_dir)
        path_main=os.path.join(dir_path,sub_dir)
        i=0
        for img_name in os.listdir(path_main):
            if i>=6:
                break
            img=cv2.imread(os.path.join(path_main,img_name))
            img=cv2.resize(img,(100,100))
            img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            X.append(img)
            i+=1
    return X
    

In [16]:
X=load_rand()

100%|██████████| 6/6 [00:00<00:00, 36.92it/s]


freshapples
freshbanana
freshoranges
rottenapples
rottenbanana
rottenoranges


In [17]:
X=np.array(X)
X.shape

(36, 100, 100, 3)

In [18]:
def show_subpot(X,title=False,Y=None):
    if X.shape[0]==36:
        f, ax= plt.subplots(6,6, figsize=(40,60))
        list_fruits=['rottenoranges', 'rottenapples', 'freshbanana', 'freshoranges', 'rottenbanana', 'freshapples']
        for i,img in enumerate(X):
            ax[i//6][i%6].imshow(img, aspect='auto')
            if title==False:
                ax[i//6][i%6].set_title(list_fruits[i//6])
            elif title and Y is not None:
                ax[i//6][i%6].set_title(Y[i])
        plt.show()
    else:
        print('Cannot plot')

In [19]:
show_subpot(X)

In [ ]:
del X

In [ ]:
def load_rottenvsfresh():
    quality=['fresh', 'rotten']
    X,Y=[],[]
    z=[]
    for cata in tqdm(os.listdir(r'C:\Users\fadel\Documents\datasets\dataset\train')):
        if quality[0] in cata:
            path_main=os.path.join(r'C:\Users\fadel\Documents\datasets\dataset\train',cata)
            for img_name in os.listdir(path_main):
                img=cv2.imread(os.path.join(path_main,img_name))
                img=cv2.resize(img,(100,100))
                img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
                z.append([img,0])
        else:
            path_main=os.path.join(r'C:\Users\fadel\Documents\datasets\dataset\train',cata)
            for img_name in os.listdir(path_main):
                img=cv2.imread(os.path.join(path_main,img_name))
                img=cv2.resize(img,(100,100))
                img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
                z.append([img,1])
    print('Shuffling your data.....')
    shuffle(z)
    for images, labels in tqdm(z):
        X.append(images);Y.append(labels)
    return X,Y

In [ ]:
X,Y=load_rottenvsfresh()

100%|██████████| 6/6 [00:30<00:00,  5.10s/it]


Shuffling your data.....


100%|██████████| 10901/10901 [00:00<00:00, 1816531.90it/s]


In [ ]:
Y=np.array(Y)
X=np.array(X)

In [ ]:
y_ser=pd.Series(Y)
y_ser.value_counts()

1    6161
0    4740
dtype: int64

In [ ]:
import keras
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Activation, Flatten, BatchNormalization
from keras.models import Sequential

In [ ]:
model = Sequential()
model.add(Conv2D())

TypeError: Conv2D.__init__() missing 2 required positional arguments: 'filters' and 'kernel_size'